In [1]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
import tweepy
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [2]:
consumer_key = 'L7dZDpDE85MYRkMdn6LRjgDpI'
consumer_secret = 'jYIJ3lufB11CeECmBUVqqS76BjXrQuYedeRqRTBq6p5uUcTsmP'
access_token = '43551104-zNapT000CHIlZAco7sMhO8FcOABi1jrF1JO6cR980'
access_token_secret = 'zRktl7Woy8ElULp8FSQq8DBw4gn81s9sKhRy4rhOjPptH'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [23]:
tweets = []
for tweet in tweepy.Cursor(api.search, q='#datascience #lima', lang='es').items(1000):
    tweets.append(tweet)

In [24]:
tweets_df = pd.DataFrame(vars(tweets[i]) for i in range(len(tweets)))

In [25]:
tweet_atts = [
'text', 'created_at', 'favorite_count',
'lang', 'retweet_count', 'source',
'in_reply_to_user_id_str', 'retweeted',
'id'
]

# subset dataframe
tweets_df = tweets_df[tweet_atts]

In [26]:
tweets_df.head()

,text,created_at,favorite_count,lang,retweet_count,source,in_reply_to_user_id_str,retweeted,id
0,RT @PeredaRea: Participando en el evento de Da...,2019-02-23 14:13:05,0,es,1,Twitter for Android,None,False,1099311186539991040
1,Participando en el evento de Data Science en l...,2019-02-23 14:11:50,1,es,1,Twitter for iPhone,None,False,1099310869874466816


In [27]:
import os
os.getcwd()

'/home/jchu/projects/dc-twitter-sa/notebooks'

In [28]:
tweets_df.to_csv('../data/tweets.csv')